In [1]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

### Get dataset list

In [2]:
import os
def get_datasets(path='../UCRArchive_2018'):
    dirs = os.listdir(path)
    return dirs

In [3]:
datasets = get_datasets()
print(datasets)

['Fungi', 'Yoga', 'ECGFiveDays', 'Wine', 'Symbols', 'SemgHandGenderCh2', 'FordB', 'SmallKitchenAppliances', 'SonyAIBORobotSurface2', 'MoteStrain', 'MixedShapesSmallTrain', 'Worms', 'UWaveGestureLibraryY', 'TwoLeadECG', 'GunPointAgeSpan', 'UMD', 'CinCECGTorso', 'PhalangesOutlinesCorrect', 'Adiac', 'DodgerLoopWeekend', 'CBF', 'PigCVP', 'DistalPhalanxTW', 'FaceFour', 'Crop', 'ElectricDevices', 'OliveOil', 'Chinatown', 'GunPointOldVersusYoung', 'HouseTwenty', 'UWaveGestureLibraryAll', 'DodgerLoopDay', 'DodgerLoopGame', 'Car', 'AllGestureWiimoteY', 'HandOutlines', 'BirdChicken', 'WordSynonyms', 'SmoothSubspace', 'SyntheticControl', 'BME', 'UWaveGestureLibraryZ', 'TwoPatterns', 'FordA', 'FiftyWords', 'UWaveGestureLibraryX', 'SemgHandMovementCh2', 'FaceAll', 'FreezerSmallTrain', 'ProximalPhalanxTW', 'PickupGestureWiimoteZ', 'DistalPhalanxOutlineAgeGroup', 'Meat', 'MixedShapesRegularTrain', 'AllGestureWiimoteX', 'FacesUCR', 'InsectEPGRegularTrain', 'NonInvasiveFetalECGThorax1', 'ECG200', 'Beef

In [4]:
dataset = datasets[6]
dataset='OSULeaf'
print(dataset)

OSULeaf


### Customize dataset

In [5]:
import numpy as np

In [6]:
data = np.load('./datasets/%s.npy' % dataset, allow_pickle=True).item()

In [7]:
freq = '1H'
custom_dataset = data['data']
print(custom_dataset.shape)
prediction_length = data['prediction_length']
context_length = custom_dataset.shape[-1] - prediction_length
print(context_length, prediction_length)
start = pd.Timestamp("01-01-2000", freq=freq)
print(start)
total_samples = custom_dataset.shape[0]

(442, 427)
342 85
2000-01-01 00:00:00


In [8]:
from gluonts.dataset.common import ListDataset

In [9]:
np.random.seed(42)
test_ratio = 0.2
test_idx = int(test_ratio * custom_dataset.shape[0]) # test size
print(test_idx)
sample_list = np.random.permutation(np.arange(total_samples)) # random shuffle
custom_train = custom_dataset[sample_list[: -test_idx]]
custom_test = custom_dataset[sample_list[-test_idx: ]]
print(custom_train.shape, custom_test.shape)

88
(354, 427) (88, 427)


In [10]:
# train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
train_ds = ListDataset([{'target': x, 'start': start}
                        for x in custom_train],
                       freq=freq)
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset([{'target': x, 'start': start}
                       for x in custom_test],
                      freq=freq)

### Prepare functions

In [11]:
batch_ratio = 0.2
num_batch = int(batch_ratio * total_samples)
print(num_batch)
num_batch = 10

88


In [12]:
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

In [13]:
def evaluate_predictor(dataset, predictor, num_samples=100, seasonality=None):
    forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset,  # test dataset
    predictor=predictor,  # predictor
    num_samples=num_samples,  # number of sample paths we want for evaluation
    )
    
    forecasts = list(forecast_it)
    tss = list(ts_it)

    # first entry of the forecast list
    forecast_entry = forecasts[0]
    # first entry of the time series list
    ts_entry = tss[0]

    print(f"Mean of the future window:\n {forecast_entry.mean}")
    print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")
    
#     plot_prob_forecasts(ts_entry, forecast_entry)
    
    if seasonality == None:
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
    else:
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9], seasonality=seasonality)
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset))
    
    p50 = agg_metrics['QuantileLoss[0.5]'] / len(forecasts)
    p90 = agg_metrics['QuantileLoss[0.9]'] / len(forecasts)
    print('p50: %.3f, p90: %.3f' % (p50, p90))
    
    print(json.dumps(agg_metrics, indent=4))

In [14]:
def evaluate(dataset, predictor, pred_length, num_samples=100, model_type=None):
    forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset,  # test dataset
    predictor=predictor,  # predictor
    num_samples=num_samples,  # number of sample paths we want for evaluation
    )
    
    y_true, y_pred = extend_data(ts_it, forecast_it, pred_length, model_type)
    clips = y_pred.shape[2]
#     p50 = compute_QL(y_pred, y_true, q=0.5)
#     p90 = compute_QL(y_pred, y_true, q=0.9)
    p50, p90 = [], []
    
    for i in range(1, clips+1, 1):
        y_true_i = y_true[:, 0, :i]
        y_pred_i = y_pred[:, 0, :i]
        p50i = compute_QL(y_pred_i, y_true_i, q=0.5).item()
        p90i = compute_QL(y_pred_i, y_true_i, q=0.9).item()
        p50.append(p50i)
        p90.append(p90i)
    return p50, p90

In [15]:
def extend_data(ts_it, forecast_it, pred_length, model_type=None):
    y_true, y_pred = [], []
    
    forecasts = list(forecast_it)
    tss = list(ts_it)
    
    for i in range(len(forecasts)):
        print('\rextending forecast... %d\%d                    ' % (i+1, len(forecasts)), end='')
        if model_type == None:
            sample = forecasts[i].samples
        elif model_type == 'mqcnn':
            sample = forecasts[i].forecast_array[5]
            sample = sample.reshape(1, sample.shape[0])
        y_pred.append(sample)
    
    for i in range(len(tss)):
        print('\rextending true... %d\%d                    ' % (i+1, len(tss)), end='')
        sample = np.array(tss[i][-pred_length:]).T
        y_true.append(sample)
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    return y_true, y_pred

In [16]:
def compute_QL(y_hat, y, q=0.5):
    ## From: https://medium.com/the-artificial-impostor/quantile-regression-part-2-6fdbc26b2629    
    QL = 2.0 * np.sum(
                np.abs(
                    (y_hat - y)
                    * ((y_hat >= y) - q)
                )
         )
    norm = np.sum(np.abs(y))
    if norm > 0:
        return QL / norm
    else:
        return QL / np.sum(np.abs(y_hat))

# Train models

### MLP

In [17]:
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator

In [18]:
fc_estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[context_length, context_length, prediction_length],
    prediction_length=prediction_length,
    context_length=context_length,
    freq=freq,
    trainer=Trainer(ctx='gpu(0)',
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch
                   )
)

In [19]:
fc_predictor = fc_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 124.10it/s, epoch=95/100, avg_epoch_loss=1.13]


In [20]:
p50, p90 = evaluate(test_ds, fc_predictor, prediction_length, num_samples=1)

extending true... 88\88                        

In [21]:
p50

[0.5454834528166026,
 0.5781789092443439,
 0.5759932948323123,
 0.6041695681798712,
 0.6189398460382576,
 0.6429983975311764,
 0.6909314670391321,
 0.7106145932217212,
 0.7429172157858083,
 0.7635354397782795,
 0.7683168276972863,
 0.7915472342706282,
 0.8367985196065876,
 0.844204113052509,
 0.8453969250070592,
 0.868109026333543,
 0.8810873553564322,
 0.8955917826725274,
 0.9192841560612438,
 0.9356479493821155,
 0.941799564420585,
 0.9537182412412593,
 0.9631011260557879,
 0.9728557652340177,
 0.9915718444474888,
 1.0016061805963372,
 1.0225531105129684,
 1.0286056698807826,
 1.0396606022543757,
 1.0503500337816758,
 1.0633070484788403,
 1.0742074645499657,
 1.090616320231481,
 1.1059130410601972,
 1.1105680794136716,
 1.1161933639145865,
 1.1169718327069793,
 1.131158653137826,
 1.1388344031515016,
 1.1411570491319416,
 1.1366219722633668,
 1.1371230993001786,
 1.1347072350087124,
 1.1360102131754175,
 1.1335311855737962,
 1.1346403036846358,
 1.1373693080893745,
 1.131025552520283

In [22]:
p90

[0.5130897530816144,
 0.47962064195428633,
 0.4522636940448961,
 0.4753095286381743,
 0.48049618730779864,
 0.5144004821865634,
 0.5418673170542455,
 0.561995565877019,
 0.5963468488703193,
 0.616601722460056,
 0.6203671511164985,
 0.6396469889364851,
 0.6517944721516737,
 0.6509462158475599,
 0.6506866709479779,
 0.6722339500745248,
 0.6864617938700631,
 0.6971591169270167,
 0.7100991086344786,
 0.7221052633695062,
 0.7375020131525557,
 0.7508827628800856,
 0.7559306355380714,
 0.7632653517971066,
 0.776747323261271,
 0.7902953623008191,
 0.8051005102497881,
 0.8077948023535538,
 0.824790468429926,
 0.8360559756887138,
 0.8477099206394281,
 0.8567196772551215,
 0.8686239839583582,
 0.8816443221971129,
 0.8910938503165828,
 0.8986915263847012,
 0.901307868249218,
 0.9176743572116899,
 0.9266779900115192,
 0.9302938254344483,
 0.9301427836861981,
 0.9359413681281944,
 0.9374763091798014,
 0.9439332884937875,
 0.9437955626021768,
 0.9475404647171141,
 0.9510338250886633,
 0.9448327570306

### GRU

In [23]:
from gluonts.model.canonical import CanonicalRNNEstimator

In [24]:
rnn_estimator = CanonicalRNNEstimator(
    prediction_length=prediction_length,
    context_length=context_length,
    freq=freq,
    cell_type='gru',
    num_layers=2,
    num_cells=context_length,
    trainer=Trainer(ctx='gpu(0)',
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch
                   )
)

In [25]:
rnn_predictor = rnn_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 18.92it/s, epoch=100/100, avg_epoch_loss=1.37]


In [26]:
p50, p90 = evaluate(test_ds, rnn_predictor, prediction_length, num_samples=1)

extending true... 88\88                        

In [27]:
p50

[1.2769109771855651,
 1.2414864365782494,
 1.3218265030806111,
 1.3091783875123917,
 1.3303747193250353,
 1.3578249880455944,
 1.3726294880293122,
 1.4013468570879812,
 1.3914232363113586,
 1.3931121207950576,
 1.3953681202950603,
 1.4096613258524846,
 1.419157207211825,
 1.4182529605688479,
 1.4174656069395917,
 1.4214068935675321,
 1.4239350756972937,
 1.4228155248295966,
 1.417425984302646,
 1.4212630921086262,
 1.4266250374985048,
 1.4317387471011946,
 1.4394360733742027,
 1.447122256440452,
 1.4646708231691044,
 1.476865971381341,
 1.492027901933137,
 1.4997986598100495,
 1.5133282238480972,
 1.5193722460893435,
 1.5255121413686854,
 1.5315314583268405,
 1.5382397565090808,
 1.5479033334078232,
 1.5516295895137961,
 1.5550382121777047,
 1.5603533080043155,
 1.5617735340614152,
 1.559069162632,
 1.5619678000056876,
 1.5648018816959404,
 1.5631250409033801,
 1.5645233009512436,
 1.5611162241145515,
 1.5603266142485583,
 1.5538010596046017,
 1.5511221584661132,
 1.5436830308605765,
 

In [28]:
p90

[1.5371982650848965,
 1.4209965222751115,
 1.5129776093387108,
 1.4562062275870042,
 1.4633478261771893,
 1.425589653713956,
 1.4228888212213195,
 1.4450864024771226,
 1.4248086699371985,
 1.3994218954732476,
 1.3862120766579706,
 1.3916592022913659,
 1.3911133861502365,
 1.3811227451030024,
 1.3785071606579202,
 1.3697112790034927,
 1.3679323852877256,
 1.3745660447865902,
 1.3553813006778082,
 1.3676135270253424,
 1.3714719710867063,
 1.3722255183032694,
 1.373943753367399,
 1.3832110919434728,
 1.3907975186516492,
 1.3903882194871635,
 1.3954928120100654,
 1.3942657616469354,
 1.3897361519416709,
 1.3830359462073145,
 1.38122222690719,
 1.3730397492544641,
 1.366701012955964,
 1.3637669067003884,
 1.353781892680507,
 1.345987101326178,
 1.341442395339909,
 1.3403965625484688,
 1.3329190022396684,
 1.3251476262089255,
 1.3192671786565013,
 1.3162582019795666,
 1.313234381921858,
 1.3068698399729766,
 1.30059194487602,
 1.2911491589377948,
 1.2828512220398847,
 1.270012065790326,
 1.2

### DeepAR

In [29]:
from gluonts.model.deepar import DeepAREstimator

In [30]:
deepar_estimator = DeepAREstimator(
    prediction_length=prediction_length,
    context_length=context_length,
    freq=freq,
    trainer=Trainer(ctx="gpu(0)",
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch
                   )
)

In [31]:
deepar_predictor = deepar_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:09<00:00,  1.09it/s, epoch=100/100, avg_epoch_loss=-1.72]


In [32]:
p50, p90 = evaluate(test_ds, deepar_predictor, prediction_length, num_samples=1)

extending true... 88\88                        

In [33]:
p50

[0.054961777384821005,
 0.08036514053969797,
 0.10834272691913374,
 0.13767722472460028,
 0.1660491408930538,
 0.19431424857967583,
 0.2223913216682361,
 0.25122593597904597,
 0.28106459620364566,
 0.31288957991281346,
 0.34475224508271624,
 0.3776271292791109,
 0.41115208992584035,
 0.44428458704665813,
 0.47684732679994046,
 0.5097756139597319,
 0.5434534452514286,
 0.5770629572733175,
 0.6097327724010513,
 0.6418248671808582,
 0.6732914350068505,
 0.7048059524252795,
 0.7366909890354448,
 0.7667289468170998,
 0.7950390549871629,
 0.8224397290046896,
 0.8482644906029662,
 0.8728553612724168,
 0.895753121798033,
 0.9176614372837314,
 0.9383072553296697,
 0.9575720387774993,
 0.9751714185913785,
 0.9907294868737477,
 1.0044999654131495,
 1.0158984677723537,
 1.0258087751231937,
 1.0342628610410114,
 1.0414477050205162,
 1.0479053156560132,
 1.0538958826098401,
 1.059538727254545,
 1.065142117043986,
 1.0706388685337764,
 1.0765860216300664,
 1.0825118467009123,
 1.0886528369653088,
 1.

In [34]:
p90

[0.06004984828079549,
 0.08486880406370136,
 0.11294326797194738,
 0.14080706793420278,
 0.1694458986842002,
 0.19763236299431583,
 0.22653040463880367,
 0.25575163695780984,
 0.2865712497793495,
 0.31948641116593407,
 0.35202697240073005,
 0.3864547981854333,
 0.42226833032448746,
 0.45793469292127875,
 0.49357479294105305,
 0.5302574412048153,
 0.5689123586766677,
 0.6084866766178211,
 0.6476539000437025,
 0.686598931446993,
 0.7251473439519805,
 0.7644276075439357,
 0.8052812713602979,
 0.8442010334179239,
 0.8805816859481121,
 0.915620026156553,
 0.9478861570922873,
 0.9778480642929173,
 1.0047597919827034,
 1.0290812667976077,
 1.051312317774612,
 1.0714654896451645,
 1.0889347336861275,
 1.103431825493558,
 1.1155942860728925,
 1.125142031057977,
 1.132874628525753,
 1.1390301643898406,
 1.1433041723791781,
 1.1469981740159525,
 1.150084711689618,
 1.1526148300180208,
 1.1545778687276345,
 1.1556577687631049,
 1.156986355471274,
 1.1579959858675422,
 1.159088269718654,
 1.1603669

### NBeats

In [35]:
from gluonts.model.n_beats import NBEATSEstimator

In [36]:
nbeats_estimator = NBEATSEstimator(
    prediction_length=prediction_length,
    context_length=context_length,
    freq=freq,
    trainer=Trainer(ctx="gpu(0)",
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch
                   )
)

In [37]:
nbeats_predictor = nbeats_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 13.69it/s, epoch=100/100, avg_epoch_loss=18.9]


In [38]:
p50, p90 = evaluate(test_ds, nbeats_predictor, prediction_length, num_samples=1)

extending true... 88\88                        

In [39]:
p50

[1.2300628393849709,
 1.2148142001779203,
 1.1914034754924259,
 1.2251289076305054,
 1.1888893708442774,
 1.18970573201853,
 1.2074353493731351,
 1.1850871055461714,
 1.168539817337631,
 1.2033978059757837,
 1.220382874791569,
 1.2117699090173046,
 1.1887108435265417,
 1.1769343760866473,
 1.5071266896287356,
 1.474797473149751,
 1.5014897545069004,
 1.489875565423299,
 1.4784993221636855,
 1.4603490099964151,
 1.4424809463511306,
 1.438290025266853,
 1.4254803940401755,
 1.412812771691178,
 1.404553874173381,
 1.3928868423007676,
 1.3862333276044563,
 1.4021880923160603,
 1.399977436837619,
 1.3957370490958247,
 1.442221022840056,
 1.4385228345321146,
 1.4367472762443627,
 1.4302161406537748,
 1.4366540685705558,
 1.4316026855125192,
 1.4273848941411658,
 1.4212183693677773,
 1.4231450767196037,
 1.427109708938289,
 1.4226159545727965,
 1.4171426651140726,
 1.4197313450708673,
 1.5208052553072435,
 1.5239953489789744,
 1.5155107442833669,
 1.4998836394804405,
 1.4916968675290647,
 1.4

In [40]:
p90

[1.2720098032170832,
 1.2115356004111975,
 1.1872320390839726,
 1.0984403382858146,
 1.1662341525468893,
 1.1643688021444611,
 1.1235880140225138,
 1.0886768331418104,
 1.0936934236881977,
 1.0669810307627727,
 1.1152647149916035,
 1.1091519112299395,
 1.100331502989697,
 1.072356817600821,
 1.086640894518506,
 1.0737277192218786,
 1.1417719772694674,
 1.167134393123013,
 1.1344586251864326,
 1.1451044720000683,
 1.1266789360789027,
 1.1425626946676222,
 1.1219786567176981,
 1.1155439760155244,
 1.1059110421774585,
 1.1037028650256036,
 1.1048878476029635,
 1.1695483975963104,
 1.188347715057424,
 1.1964041098877285,
 1.3081705335440195,
 1.3029570687616843,
 1.2826224267340507,
 1.2661482606304968,
 1.2546577622739419,
 1.2593419727286255,
 1.264303038175473,
 1.255066391605751,
 1.2457593035188965,
 1.2689431235231496,
 1.2658249351801976,
 1.249190969221121,
 1.2304620086971654,
 1.227492187229631,
 1.255740491146969,
 1.2481142671855634,
 1.23741010948397,
 1.225372507865988,
 1.21

### MQCNN

In [41]:
from gluonts.model.seq2seq import MQCNNEstimator

In [59]:
mqcnn_estimator = MQCNNEstimator(
    prediction_length=prediction_length,
    context_length=context_length,
    freq=freq,
    trainer=Trainer(ctx="gpu(0)",
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch
                   )
)

In [60]:
mqcnn_predictor = mqcnn_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 23.11it/s, epoch=100/100, avg_epoch_loss=0.0535]


In [61]:
p50, p90 = evaluate(test_ds, mqcnn_predictor, prediction_length, num_samples=1, model_type='mqcnn')

extending true... 88\88                        

In [62]:
p50

[0.054414006940668565,
 0.06815339881815749,
 0.08429277240168362,
 0.10511620026180499,
 0.12634500609486574,
 0.14919295203673566,
 0.17281940613763466,
 0.19792635394268104,
 0.22322956656348056,
 0.2486755196789812,
 0.27428782942214397,
 0.29947745779028007,
 0.3244121577698238,
 0.3482360287129327,
 0.37074822148113923,
 0.39362079751318363,
 0.4161462666769308,
 0.437557953821946,
 0.45931049260466866,
 0.4801089784140936,
 0.49992076029119853,
 0.5186956173394167,
 0.5367913265819113,
 0.5540600894250247,
 0.5705472534145622,
 0.5861498510710593,
 0.6005479074574365,
 0.6142340971882247,
 0.626241598403252,
 0.639168930572298,
 0.6510330705963148,
 0.6616724566479893,
 0.6727188762950865,
 0.6828619055589576,
 0.6924517312877752,
 0.701282249371738,
 0.7095625913273533,
 0.7171174470434627,
 0.7246644138895005,
 0.7309359409820979,
 0.736858564912654,
 0.7421284120352428,
 0.7477024065161846,
 0.7529513736160196,
 0.7570726108564999,
 0.7611748651747834,
 0.7652277283639222,
 0

In [63]:
p90

[0.08040178673753828,
 0.09194767481625325,
 0.10330361500947782,
 0.12109104078196338,
 0.14017340145898216,
 0.15635918759777215,
 0.17289914445083024,
 0.1922404072580289,
 0.21206172324659672,
 0.23249287152233178,
 0.251596704957127,
 0.2719071717189358,
 0.289562878937608,
 0.306113895445186,
 0.3215258419478187,
 0.33650118102570453,
 0.3518298274641916,
 0.3659390921093768,
 0.3796497215503766,
 0.3927905125853158,
 0.40639278112147365,
 0.4179961823949336,
 0.42929445738038463,
 0.4408765364204203,
 0.45141505684883954,
 0.4610801472533154,
 0.46970606978312374,
 0.47673227014386294,
 0.4829628484555973,
 0.48769067963514345,
 0.49328841940062707,
 0.5001336024254417,
 0.504534909533215,
 0.5095853965495833,
 0.5159014701922503,
 0.520545324195965,
 0.5261680124912419,
 0.5313907102813507,
 0.5355233818888628,
 0.5406699457119065,
 0.5448993963724624,
 0.5491842442059468,
 0.5529444258139963,
 0.5564010902118062,
 0.5606075009354761,
 0.5647497923044229,
 0.5703433787454014,
 

### Transformer

In [47]:
from gluonts.model.transformer import TransformerEstimator

In [48]:
transformer_estimator = TransformerEstimator(
    prediction_length=prediction_length,
    context_length=context_length,
    freq=freq,
    trainer=Trainer(ctx="gpu(0)",
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch,
                   )
)

In [49]:
transformer_predictor = transformer_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 33.41it/s, epoch=100/100, avg_epoch_loss=-1.06]


In [50]:
p50, p90 = evaluate(test_ds, transformer_predictor, prediction_length, num_samples=1)

extending true... 88\88                        

In [51]:
p50

[0.09331094013298755,
 0.13035359846661992,
 0.1682622426802261,
 0.2059703778792186,
 0.2532828331944378,
 0.30347079592702375,
 0.3569751096805315,
 0.4086669728792355,
 0.4631604284495732,
 0.5214986875223645,
 0.5804896936665497,
 0.6417647111867809,
 0.7021445263581709,
 0.7583874519332365,
 0.8137659534028686,
 0.8692072279403462,
 0.9221921209352347,
 0.9765597636383146,
 1.03022427700643,
 1.0831049459068474,
 1.1323315895985016,
 1.1816277216278204,
 1.2289425654417292,
 1.2727317346057738,
 1.311926009041041,
 1.3482673702539767,
 1.378815655650337,
 1.4083876926481618,
 1.4338535708451077,
 1.4560072002269042,
 1.4753668481194766,
 1.491872074306074,
 1.5074055814390355,
 1.5204223129912973,
 1.532244670281403,
 1.542834535020237,
 1.5508030291782753,
 1.5585355987405975,
 1.5648132438319657,
 1.5690242645131192,
 1.5721562959007427,
 1.574170909439295,
 1.5754135532356084,
 1.5744078371717949,
 1.5724122782428103,
 1.5694076315084653,
 1.566708518663133,
 1.5644936110399372

In [52]:
p90

[0.1297531401038646,
 0.18399432129058432,
 0.23674441379705427,
 0.2961136907788997,
 0.37039306531827054,
 0.45153415185699997,
 0.5404802239652593,
 0.6289885736674802,
 0.7225146210855982,
 0.8216079670740013,
 0.9211988117318568,
 1.0232588407912921,
 1.1250850888628852,
 1.2220243926681489,
 1.3173318812814603,
 1.4132792127507303,
 1.5049291505624354,
 1.600073579797921,
 1.6929130580326142,
 1.7842850919950803,
 1.8693938763571518,
 1.9547343156018773,
 2.0366702386052706,
 2.1121258040906037,
 2.1793205023497073,
 2.241222285508919,
 2.291845404019278,
 2.3383712657061664,
 2.376022451559143,
 2.406538638239102,
 2.4307013696429487,
 2.4483278923328564,
 2.461696142093299,
 2.468758812743557,
 2.4714093379454583,
 2.470754125193662,
 2.464904997020855,
 2.456787289039957,
 2.4450787332150727,
 2.430123915251753,
 2.41279528422953,
 2.39406104231321,
 2.374254910109767,
 2.352586047406814,
 2.331126795591975,
 2.3097681904162157,
 2.290123605743247,
 2.2729837324159385,
 2.2586

### WaveNet

In [53]:
from gluonts.model.wavenet import WaveNetEstimator

In [54]:
wavenet_estimator = WaveNetEstimator(
    prediction_length=prediction_length,
    freq=freq,
    trainer=Trainer(ctx="gpu(0)",
                    epochs=100,
                    learning_rate=1e-3,
                    hybridize=False,
                    num_batches_per_epoch=num_batch
                   )
)

In [55]:
wavenet_predictor = wavenet_estimator.train(train_ds)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 32.70it/s, epoch=100/100, avg_epoch_loss=1.96]


In [56]:
p50, p90 = evaluate(test_ds, wavenet_predictor, prediction_length, num_samples=1)

extending true... 88\88                        

In [57]:
p50

[0.5815842378864743,
 0.6223323841514024,
 0.6676222848498609,
 0.7108634398050562,
 0.7549703263252131,
 0.7978680428755907,
 0.8382319708453617,
 0.8745486305835416,
 0.90700192944871,
 0.9387577290303936,
 0.9725485234817708,
 0.9996986159989486,
 1.0276973136729268,
 1.0548471943092923,
 1.0851954395907233,
 1.1135443223399717,
 1.1383274650088955,
 1.1632574933565742,
 1.1850785464627367,
 1.2054683460344713,
 1.2247092990348118,
 1.2442017325320684,
 1.2626250083241268,
 1.2829089904849604,
 1.302064084115301,
 1.318210978752912,
 1.3345240481033167,
 1.3506561210520611,
 1.3646832423996673,
 1.3789963165680326,
 1.3912928524558887,
 1.403521716960655,
 1.4148224654770205,
 1.4264850372922298,
 1.4363467602292965,
 1.4456333110434132,
 1.4528918941184035,
 1.458955345818951,
 1.4636733211613553,
 1.467404646088236,
 1.4706231565122445,
 1.4727078072916195,
 1.4742988500595575,
 1.4761166650185977,
 1.4778774153654812,
 1.4790355843455079,
 1.4826844304770903,
 1.4840661875917456,

In [58]:
p90

[0.1432833946469306,
 0.16460327060405067,
 0.1903316726549513,
 0.2120102252481227,
 0.23107188507773926,
 0.2516840650537137,
 0.2685605524140866,
 0.28325537845003224,
 0.29455350591851737,
 0.30742448815064194,
 0.3196201719896566,
 0.3293819473548884,
 0.33866703425415745,
 0.3478631408101346,
 0.35815507924699996,
 0.36823044992886356,
 0.3778143963722062,
 0.3876041974940752,
 0.396658536384375,
 0.40559757033465554,
 0.4140131699231009,
 0.4222759569679643,
 0.4300072031735558,
 0.438062448263342,
 0.44535553257163857,
 0.4517669857719272,
 0.457984243838971,
 0.4641776612611549,
 0.47006581407572756,
 0.4758833598740176,
 0.4813757180816345,
 0.4871977870423129,
 0.4930618843277739,
 0.4989533142751664,
 0.504308853929337,
 0.5095810531961176,
 0.5144737728719879,
 0.5187259221667972,
 0.5224380873393887,
 0.5259163565585205,
 0.5292105721382305,
 0.5324061512178013,
 0.5352523792607203,
 0.5381997988481272,
 0.5409763968027196,
 0.5432953200039286,
 0.5456826673670815,
 0.547